In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from time import time

import bioio_nd2
import numpy as np
from bioio import BioImage
from cellpose.contrib.distributed_segmentation import (
    distributed_eval,
    numpy_array_to_zarr,
    process_block,
)

from iob_ia.utils import segment, visualise

In [6]:
import zarr

path_base = ""

z = zarr.open(path_base + 'test_zarr.zarr')
s = zarr.open(path_base + 'test_zarr_segmented.zarr')
#visualise.add_image(z, name='img')
#visualise.add_labels(s, name='seg')

import dask as da
start = time()
zc = da.array.from_zarr(path_base + 'test_zarr.zarr').compute()
sc = da.array.from_zarr(path_base + 'test_zarr_segmented.zarr').compute()
print('took:', time()-start)
#print(zc)

In [5]:
from skimage.measure import regionprops_table

start = time()
props = regionprops_table(s, z, properties=('label', 'area'))
print('took:', time()-start)
print(props)

took: 2815.7092139720917
{'label': array([    1,     2,     3, ..., 34246, 34247, 34248]), 'area': array([ 533., 1426., 7764., ...,   56.,   74.,  104.])}


In [7]:
print(len(props['label']))

34248


C:\Users\Microscopy\.conda\envs\iob-ia\lib\site-packages\cupy\_environment.py:217: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


!! Still uses the cpu not gpu

In [2]:
from cucim.skimage.measure import regionprops_table as r_t

In [7]:
start = time()
props_cupy = r_t(s, z, properties=('label', 'area'))
print('took:', time()-start)
print(props_cupy)

took: 2789.7874755859375
{'label': array([    1,     2,     3, ..., 34246, 34247, 34248]), 'area': array([ 533., 1426., 7764., ...,   56.,   74.,  104.])}


## Create a zarr file (1 channel)

In [5]:
path_base = 'G:/20250211_MarcDu/'
path_nd2 = path_base + '24 C3 10X Large Image Z.nd2'

In [ ]:
img_nd2 = BioImage(path_nd2, reader=bioio_nd2.Reader)
print(img_nd2.shape)


In [ ]:
start = time()
dapi = np.squeeze(img_nd2.data[0])[0]
print(dapi.shape)
print('took:', time() - start)

In [ ]:
# check the file size and RAM
from sys import getsizeof

import psutil

print(getsizeof(dapi))
print(dapi.nbytes)
print(psutil.virtual_memory())

In [ ]:
#del img_nd2
psutil.virtual_memory().used/psutil.virtual_memory().total

In [ ]:
print(psutil.virtual_memory().total / 1024**3)
print(psutil.virtual_memory().total *1.0**-9)

In [ ]:
start = time()
chunks=(63,2048,2048)
z = numpy_array_to_zarr(path_base + 'test_zarr.zarr', np.squeeze(img_nd2.data)[0], chunks=chunks)
print(z.shape)
print('took:', time() - start)

#data_numpy = tifffile.imread('/path/to/image.tiff')
#data_zarr = numpy_array_to_zarr('/path/to/output.zarr', data_numpy, chunks=(256, 256, 256))
#del data_numpy  # assumption is data is large, don't keep in memory copy around

In [ ]:
del dapi

In [ ]:
# predict
model_kwargs = {'gpu':True, 'model_type':'cyto3'}
eval_kwargs = {
    'diameter':12,
    'z_axis':0,
    'channels':[0,0],
    'do_3D':True,
}
cluster_kwargs = {
    'n_workers':1,  # for 1 GPU
    'ncpus':24,
    'memory_limit':'196GB',
    'threads_per_worker':1,
}



In [ ]:
# Test run on single block
start = time()

starts = (0, 4500, 2500)
blocksize = chunks
overlap = 0
crop = tuple(slice(s-overlap, s+b+overlap) for s, b in zip(starts, blocksize))
#crop = z[:, starts[1]:starts[1]+blocksize[1], starts[2]:starts[2]+blocksize[2]]
print('crop tuple=', crop)

segments, boxes, box_ids = process_block(
    block_index=(0,0,0), # when test_mode=True this is just a dummy value
    crop=crop,
    input_zarr=z,
    model_kwargs=model_kwargs,
    eval_kwargs=eval_kwargs,
    blocksize=blocksize,
    overlap=overlap,
    output_zarr=None,
    test_mode=True,
)

print('took:', time() - start)

In [ ]:
# visualise the test
img_crop = z[:, starts[1]:starts[1]+blocksize[1], starts[2]:starts[2]+blocksize[2]]
visualise.add_image(img_crop, name='image_crop', scale=img_nd2.physical_pixel_sizes)
visualise.add_labels(segments, name='test_seg', scale=img_nd2.physical_pixel_sizes)

In [ ]:
# Full image run
start = time()

mask, boxes = distributed_eval(
    input_zarr=z,
    blocksize=chunks,
    write_path=path_base + 'test_zarr_segmented.zarr',
    model_kwargs=model_kwargs,
    eval_kwargs=eval_kwargs,
    cluster_kwargs=cluster_kwargs,
)

print('took:', time() - start)

In [ ]:
mask.shape

In [ ]:
# add multi channel nd2
visualise.add_multichannel_image(
    img=np.squeeze(img_nd2.xarray_dask_data), name='image',
    channel_names=img_nd2.channel_names, scale=img_nd2.physical_pixel_sizes
)
visualise.add_labels(
    mask, name='nuclei', scale=img_nd2.physical_pixel_sizes,
)

# To predict zarr arrays I need to use:

see here: https://cellpose.readthedocs.io/en/latest/distributed.html#examples

```python
from cellpose.contrib.distributed_segmentation import distributed_eval

 # parameterize cellpose however you like
 model_kwargs = {'gpu':True, 'model_type':'cyto3'}  # can also use 'pretrained_model'
 eval_kwargs = {'diameter':30,
                'z_axis':0,
                'channels':[0,0],
                'do_3D':True,
 }

 # define compute resources for local workstation
 cluster_kwargs = {
     'n_workers':1,    # if you only have 1 gpu, then 1 worker is the right choice
     'ncpus':8,
     'memory_limit':'64GB',
     'threads_per_worker':1,
 }

 # run segmentation
 # outputs:
 #     segments: zarr array containing labels
 #     boxes: list of bounding boxes around all labels (very useful for navigating big data)
 segments, boxes = distributed_eval(
     input_zarr=large_zarr_array,
     blocksize=(256, 256, 256),
     write_path='/where/zarr/array/containing/results/will/be/written.zarr',
     model_kwargs=model_kwargs,
     eval_kwargs=eval_kwargs,
     cluster_kwargs=cluster_kwargs,
 )
```

In [ ]:
z.squeeze()

import zarr
store = zarr.storage.MemoryStore()

dapi = None

z = zarr.create(shape=dapi.shape, chunks=(1,500,500), store=store, dtype=img_nd2.dtype)
print(z.squeeze())


In [ ]:
dapi = None
mask_cp = segment.segment_3d_cellpose(
    dapi
)
print(mask_cp.shape)

In [ ]:
visualise.add_multichannel_image(
    img=img_nd2.get_dask_data(),
    name='image',
    channel_names=img_nd2.channel_names,
    scale=img_nd2.physical_pixel_sizes
)
visualise.add_labels(
    img=mask_cp,
    name='nuclei',
    scale=img_nd2.physical_pixel_sizes
)